In [1]:
metrics = {
    "profitability": {
        "class_weight": 0.35,
        "weights": [1.0], 
        "metrics": {
            "oper_margin": {
                "lower_is_better": False,
                "thresholds": [
                    (0.4, float("inf")),
                    (0.35, 0.4),
                    (0.3, 0.35),
                    (0.25, 0.3),
                    (0.2, 0.25),
                    (0.15, 0.2),
                    (0.1, 0.15),
                    (0.05, 0.1),
                    (float("-inf"), 0.05),
                ],
            }
        },
    },
    "leverage_coverage": {
        "class_weight": 0.65,
        "weights": [0.4, 0.3, 0.3],
        "metrics": {
            "debt_to_equity": {
                "lower_is_better": True,
                "thresholds": [
                    (0, 0.4),
                    (0.4, 0.8),
                    (0.8, 1.2),
                    (1.2, 1.6),
                    (1.6, 2),
                    (2, 2.5),
                    (2.5, 3),
                    (3, 4),
                    (4, float("inf")),
                ],
            },
            "tot_debt_to_ebitda": {
                "lower_is_better": True,
                "thresholds": [
                    (0, float("-inf")),
                    (0.4, 0.8),
                    (0.8, 1.2),
                    (1.2, 1.6),
                    (1.6, 2),
                    (2, 2.5),
                    (2.5, 3),
                    (3, 4),
                    (4, float("inf")),
                ],
            },
            "ebitda_to_tot_int_exp": {
                "lower_is_better": False,
                "thresholds": [
                    (25, float("inf")),
                    (20, 25),
                    (15, 20),
                    (10, 15),
                    (5, 10),
                    (3, 5),
                    (1, 3),
                    (0, 1),
                    (float("-inf"), 0),
                ],
            },
        },
    },
}


In [4]:
class CreditRatingCalculator:
    def __init__(self, metrics):
        self.metrics = metrics
        
    def _calculate_metric_score(self, metric, thresholds, inverse):
        for score, (lower, upper) in enumerate(thresholds, start=1):
            if (inverse and metric <= upper) or (not inverse and metric >= lower):
                return score
        return len(thresholds) // 2 # else return the middle score

    def _calculate_category_score(self, category_metrics, ratios):
        total_weighted_score = 0

        for metric, weight in zip(
            category_metrics["metrics"].items(), category_metrics["weights"]
        ):
            metric_name, metric_data = metric
            value = ratios[metric_name]
            score = self._calculate_metric_score(
                value, metric_data["thresholds"], metric_data["lower_is_better"]
            )
            total_weighted_score += score * weight

        return total_weighted_score

    def _calculate_scores(self, ratios):
        scores = {}
        for category, category_data in self.metrics.items():
            category_score = self._calculate_category_score(category_data, ratios)
            scores[category] = category_score
        return scores

    def _calculate_weighted_score(self, scores):
        weights = {
            category: category_data["class_weight"]
            for category, category_data in self.metrics.items()
        }
        return sum(scores[category] * weight for category, weight in weights.items())

    def _determine_credit_rating(self, weighted_score):
        credit_ratings = [
            (1.5, "Aaa"),
            (2.5, "Aa"),
            (3.5, "A"),
            (4.5, "Baa"),
            (5.5, "Ba"),
            (6.5, "B"),
            (7.5, "Caa"),
            (8.5, "Ca"),
            (float("inf"), "C"),
        ]

        for threshold, rating in credit_ratings:
            if weighted_score < threshold:
                return rating

    def calculate_credit_rating(self, ratios):
        self.scores = self._calculate_scores(ratios)
        self.credit_score = self._calculate_weighted_score(self.scores)
        self.credit_rating = self._determine_credit_rating(self.credit_score)

In [6]:
ratios = {
    'oper_margin': 6.0,
    'debt_to_equity': 0.2,
    'tot_debt_to_ebitda': 0.5,
    'ebitda_to_tot_int_exp': 0.6,
}

model = CreditRatingCalculator(metrics)
model.calculate_credit_rating(ratios)
print(f"Class Scoring: {model.scores}")
print(f"Credit Score: {model.credit_score}")
print(f"Credit Rating: {model.credit_rating}")

Class Scoring: {'profitability': 1.0, 'leverage_coverage': 3.4}
Credit Score: 2.56
Credit Rating: A
